In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Step 1: Data Preprocessing
# Load the dataset
df = pd.read_csv("/content/fashion_products.csv")

In [3]:
# Clean the dataset by dropping irrelevant columns
df = df.drop(columns=["Product Name"])

In [4]:
# Handle missing values
df = df.dropna()

In [6]:
# Encode categorical features
categorical_features = ["Brand", "Category", "Color", "Size"]
label_encoders = {}
for feature in categorical_features:
    label_encoders[feature] = LabelEncoder()
    df[feature] = label_encoders[feature].fit_transform(df[feature])

In [7]:
# Step 2: Collaborative Filtering
# Create a user-item interaction matrix
interaction_matrix = df.pivot(index='User ID', columns='Product ID', values='Rating').fillna(0)

# Apply collaborative filtering (item-based)
item_similarity = cosine_similarity(interaction_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, index=interaction_matrix.columns, columns=interaction_matrix.columns)

In [8]:
# Step 3: Content-Based Filtering
# Create a product-feature matrix
product_features = df[categorical_features].values
product_similarity = cosine_similarity(product_features)
product_similarity_df = pd.DataFrame(product_similarity, index=df["Product ID"], columns=df["Product ID"])

In [9]:
# Step 4: Hybrid Recommender System
def hybrid_recommend(user_id, top_n=5, collaborative_weight=0.5):
    # Collaborative filtering recommendation
    user_ratings = interaction_matrix.loc[user_id]
    similar_items = pd.Series()
    for i, rating in user_ratings.iteritems():
        similar_items = similar_items.append(item_similarity_df[i] * rating)
    similar_items = similar_items.sort_values(ascending=False)

    # Content-based filtering recommendation
    user_interactions = interaction_matrix.loc[user_id]
    user_liked_items = user_interactions[user_interactions > 0].index
    content_based_recommendations = pd.Series()
    for item in user_liked_items:
        content_based_recommendations = content_based_recommendations.append(product_similarity_df[item])
    content_based_recommendations = content_based_recommendations.groupby(content_based_recommendations.index).mean()
    content_based_recommendations = content_based_recommendations.sort_values(ascending=False)

    # Combine recommendations
    hybrid_recommendations = (collaborative_weight * similar_items + (1 - collaborative_weight) * content_based_recommendations).sort_values(ascending=False)

    return hybrid_recommendations[:top_n]

In [13]:
# Step 5: Example usage and evaluation
target_user_id = 1
recommendations = hybrid_recommend(target_user_id)
print("Hybrid Recommendations for User", target_user_id)
print(recommendations)

<ipython-input-9-bb37b2a1a649>:5: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  similar_items = pd.Series()
<ipython-input-9-bb37b2a1a649>:6: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, rating in user_ratings.iteritems():
<ipython-input-9-bb37b2a1a649>:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_items = similar_items.append(item_similarity_df[i] * rating)
<ipython-input-9-bb37b2a1a649>:7: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similar_items = similar_items.append(item_similarity_df[i] * rating)
<ipython-input-9-bb37b2a1a649>:7: FutureWarning: The series.append method is deprecated and will be removed from panda

Hybrid Recommendations for User 1
964    2.766348
947    2.760836
879    2.760510
515    2.758719
479    2.743766
dtype: float64


In [12]:
print(df.head())


   User ID  Product ID  Brand  Category  Price    Rating  Color  Size
0       19           1      0         1     40  1.043159      0     3
1       97           2      2         2     82  4.026416      0     0
2       25           3      0         2     44  3.337938      5     3
3       57           4      4         1     23  1.049523      4     2
4       79           5      0         1     79  4.302773      0     1
